# Data lab 4 - Portfolio Performance Attribution and Factor Model

#### Note 1: Review what you have learned in Data labs, DataCamp assignments, and in-class sample code.
#### Note 2: Chapter "Performance Attribution" of DataCamp course "Introduction to Portfolio Analysis in Python"  is a useful reference.
#### Note 3: This note serves as a guide. You are free to tinker with it! 

### 1. Revisit the all-weather portfolio you crafted. Create the maximum Sharpe portfolio's daily return dataframe and then merge it with Fama French's five return factors.

#### If you have attempted the optional bonus, why not include the portfolios with L2 regularization and Black-Litterman model too.

In [17]:
# Import the necessary packages
import yfinance as yf
import datetime as dt 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
# Pick a list of stocks that form your all-weather portfolio
symbols_list = ["AAPL", "PG", "NEE", "UNH", "UNP", "INFY", "COST", "MDT", "AMT", "V"]
# Draw data for the past five years
start = dt.datetime(2017,10,1)
end = dt.datetime(2022,10,1)
data = yf.download(symbols_list, start=start, end=end)
#Illustrative, change into yours!
data.head(5)




[*********************100%***********************]  10 of 10 completed


Adj Close                                                          \
                 AAPL         AMT        COST      INFY        MDT        NEE   
Date                                                                            
2017-10-02  36.493935  122.773071  154.681854  6.408523  69.926819  32.768734   
2017-10-03  36.652897  123.496033  153.926559  6.351772  69.980576  32.802231   
2017-10-04  36.415634  125.466019  154.047821  6.334310  70.446457  33.177387   
2017-10-05  36.868813  125.836533  155.782120  6.338676  71.234879  32.996506   
2017-10-06  36.847462  125.710022  146.476410  6.386695  71.503639  32.931744   

                                                           ...    Volume  \
                   PG         UNH         UNP           V  ...      AAPL   
Date                                                       ...             
2017-10-02  80.058449  183.454269  103.758804  102.081459  ...  74795200   
2017-10-03  80.363777  184.559967  103.425720  102.226669  ...  64921200   
2017-10-04  80.625504  186.278915  102.417473  101.955582  ...  80655200   
2017-10-05  80.285271  183.881668  102.642548  102.855965  ...  85135200   
2017-10-06  80.547020  184.030380  102.579514  103.330368  ...  69630400   

                                                                             \
                AMT      COST      INFY      MDT      NEE       PG      UNH   
Date                                                                          
2017-10-02  1513400   2267100  10237800  4312900  4581200  7302400  2893600   
2017-10-03  1885800   2839400  15808200  5866600  4008400  5909900  2533800   
2017-10-04  1827000   1918600  10879200  5735200  4390400  6140700  2286400   
2017-10-05  1965400   4715400   4980800  6323200  6817600  6912800  4069000   
2017-10-06  1377200  18001200   4455800  3248500  5110800  5241900  3361900   

                              
                UNP        V  
Date                          
2017-10-02  3222200  4885600  
2017-10-03  2783900  4411100  
2017-10-04  3229600  4146800  
2017-10-05  2811500  4396100  
2017-10-06  2200100  4587000  

[5 rows x 60 columns]

In [19]:
# Keep only the adjusted close in the dataframe
# Note that the date is in the index
price = data["Adj Close"]

In [20]:
# Calculate return using method pct_change
# Find out more about .pct_change with help! 
daily_return = price.pct_change()
daily_return.head(5)

,AAPL,AMT,COST,INFY,MDT,NEE,PG,UNH,UNP,V
Date,,,,,,,,,,
2017-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-03,0.004356,0.005889,-0.004883,-0.008856,0.000769,0.001022,0.003814,0.006027,-0.003210,0.001422
2017-10-04,-0.006473,0.015952,0.000788,-0.002749,0.006657,0.011437,0.003257,0.009314,-0.009749,-0.002652
2017-10-05,0.012445,0.002953,0.011258,0.000689,0.011192,-0.005452,-0.004220,-0.012869,0.002198,0.008831
2017-10-06,-0.000579,-0.001005,-0.059735,0.007576,0.003773,-0.001963,0.003260,0.000809,-0.000614,0.004612


In [21]:
# function that takes portfolio weights and creates a time-series of daily portfolio returns
def portfolio_return_series(daily_returns, weights):
    
    ''' 
    INPUTS
    daily returns: dataframe of daily returns. Each ticker column contains the series of daily returns for the ticker
    weights: numpy array of the portfolio weight on each ticker (sorted in ascending order)
    
    OUTPUTS
    portfolio_daily_returns: the portfolio return series given the weights
    
    '''    
    
    # Create portfolio daily returns
    portfolio_daily_returns = daily_returns.dot(weights)
    return portfolio_daily_returns

In [22]:
## Install PyPortfolioOpt package
# !pip install PyPortfolioOpt

# Import the packages 
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

The Maximum Sharpe Portfolio

In [24]:
# Calculate expected returns mu 
# Calculate expected returns mu 
mu = expected_returns.mean_historical_return(price)

# Calculate the covariance matrix S
cov_matrix_d = daily_return.cov()

# Obtain the efficient frontier
ef = EfficientFrontier(mu, cov_matrix_d)

# Calculate weights for the maximum Sharpe ratio portfolio
raw_weight_maxsharpe = ef.max_sharpe()
clean_weight_maxsharpe = ef.clean_weights()
# Inspect the calculated weights
print("Raw weight: ", raw_weight_maxsharpe)
print("Cleaned weight: ", clean_weight_maxsharpe)


Raw weight:  OrderedDict([('AAPL', 0.218444565564576), ('AMT', 0.0), ('COST', 0.4850353799649264), ('INFY', 0.1191442912127117), ('MDT', 0.0), ('NEE', 0.104321223127052), ('PG', 0.0), ('UNH', 0.0730545401307338), ('UNP', 0.0), ('V', 0.0)])
Cleaned weight:  OrderedDict([('AAPL', 0.21844), ('AMT', 0.0), ('COST', 0.48504), ('INFY', 0.11914), ('MDT', 0.0), ('NEE', 0.10432), ('PG', 0.0), ('UNH', 0.07305), ('UNP', 0.0), ('V', 0.0)])


The minimum volatility portfolio

In [25]:
# Obtain the efficient frontier
mu = expected_returns.mean_historical_return(price)
sigma = risk_models.sample_cov(price)
ef = EfficientFrontier(mu, sigma)
# Calculate weights for the minimum volatility portfolio
raw_weight_minvol = ef.min_volatility()
clean_weight_minvol = ef.clean_weights()
print("Raw weight: ", raw_weight_minvol)
print("Cleaned weight: ", clean_weight_minvol)

Raw weight:  OrderedDict([('AAPL', 0.0), ('AMT', 0.006205920173865), ('COST', 0.2315996620595008), ('INFY', 0.1171307038590641), ('MDT', 0.1627015261429157), ('NEE', 0.0737659342421997), ('PG', 0.3512166428574474), ('UNH', 0.0), ('UNP', 0.0573796106650074), ('V', 0.0)])
Cleaned weight:  OrderedDict([('AAPL', 0.0), ('AMT', 0.00621), ('COST', 0.2316), ('INFY', 0.11713), ('MDT', 0.1627), ('NEE', 0.07377), ('PG', 0.35122), ('UNH', 0.0), ('UNP', 0.05738), ('V', 0.0)])


In [68]:
# 1. Daily portfolio returns for the equally-weighted portfolio
equal_weight = np.repeat(0.1, 10)
equal_weight_return = portfolio_return_series(daily_return, equal_weight)
# Extract the first element from the function output for daily returns
equal_weight_first_element = equal_weight_return.iloc[1]

# 2. Daily portfolio returns for the maximum Sharpe portfolio
# Extract the first element from the function output for daily returns
clean_weight_maxsharpe_list = list(clean_weight_maxsharpe.values())
clean_weight_maxsharpe_array = np.array(clean_weight_maxsharpe_list)
max_sharpe_daily_return = portfolio_return_series(daily_return, clean_weight_maxsharpe_array)
max_sharpe_first_element = max_sharpe_daily_return.iloc[1]

# 3. Daily portfolio returns for the minimum volatility portfolio
# Extract the first element from the function output for daily returns
clean_weight_minvol_list = list(clean_weight_minvol.values())
clean_weight_minvol_array = np.array(clean_weight_minvol_list)
minvol_daily_return = portfolio_return_series(daily_return, clean_weight_minvol_array)
minvol_first_element = minvol_daily_return.iloc[1]



# Merge the three series side-by-side into a dataframe
# Note the index is date
portfolio_returns = pd.concat([equal_weight_return, max_sharpe_daily_return, minvol_daily_return], axis = 1)
# Rename column names
portfolio_returns = portfolio_returns.rename(columns = {0: "Equal_Weight_Return", 1: "MaxSharpe_Return", 2: "Minvol_Return"})
print(portfolio_returns)


            Equal_Weight_Return  MaxSharpe_Return  Minvol_Return
Date                                                            
2017-10-02                  NaN               NaN            NaN
2017-10-03             0.000635         -0.001925      -0.000776
2017-10-04             0.002578          0.000514       0.002471
2017-10-05             0.002702          0.006752       0.002769
2017-10-06            -0.004387         -0.028344      -0.011375
...                         ...               ...            ...
2022-09-26            -0.007116          0.013044       0.002884
2022-09-27            -0.006186          0.000040      -0.011913
2022-09-28             0.010785          0.012098       0.013008
2022-09-29            -0.018959         -0.025206      -0.018316
2022-09-30            -0.013516         -0.015442      -0.015124

[1259 rows x 3 columns]


In [69]:
# Inspect the last five observations of the portfolio_returns dataframe
portfolio_returns.tail(5)

,Equal_Weight_Return,MaxSharpe_Return,Minvol_Return
Date,,,
2022-09-26,-0.007116,0.013044,0.002884
2022-09-27,-0.006186,0.000040,-0.011913
2022-09-28,0.010785,0.012098,0.013008
2022-09-29,-0.018959,-0.025206,-0.018316
2022-09-30,-0.013516,-0.015442,-0.015124


In [ ]:
# Read the csv file with factor returns with pd.read_csv()
# Note: place the file where your Jupyter notebook is


# Divide all factor returns by 100
# Consistent with how we calculate portfolio returns


This file is from http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html#Research and is created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202208 CRSP database. The 1-month TBill return is from Ibbotson and Associates. For more details on the factor returns, please read http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html

In [ ]:
# Inspect the dataframe with info()


In [ ]:
# Inspect the last five observations


In [ ]:
# Merge portfolio daily returns with factor returns


In [ ]:
# Inspect the last five observations


In [ ]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
portfolio_return = ['portfolio_ew', 'portfolio_maxsharpe', 'portfolio_minvol']

# Print correlation table. Hint use .corr()


### 2. Examine visually the correlation between portfolio and factor returns

#### Hint: Write functions for repetitive lines of codes 

In [ ]:
# function that takes portfolio and factor returns and creates 20-day rolling correlation with a return factor
def portfolio_factor_correlation(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    plot the 20-day rolling correlation between a portfolio and a factor return
    
    '''    
    
    for factor in factors:
        
        # Calculate 20-day rolling correlation with the market


        # Plot the correlation between a portfolio and factor returns


    return 

IndentationError: expected an indented block (4268343362.py, line 22)

In [ ]:
# Call the function portfolio_factor_correlation for the equally weighted portfolio


In [ ]:
# Call the function portfolio_factor_correlation for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_factor_correlation for the minimum volatility portfolio


### 3. Regress the portfolio return on each factor and assess the portfolio's sensitivity to each factor.

#### For the curious, optional challenge, how do you test whether the intercept (i.e., alpha) is significantly different from the risk-free rate for a single-factor regression?
#### Hint: Write functions for repetitive lines of codes

In [ ]:
# Import the ols function
import statsmodels.api as sm

# function that takes portfolio and factor returns and run a regression of portfolio return on a factor return
# it reports the portfolio sensitivity to a return factor
def portfolio_factor_sensitivity(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    regression result of a portfolio on a return factor
    
    '''    
    for factor in factors:
        
        # Create the regression model object

        # Fit the model

        # Print the parameters of the fitted model

        # Optional challenge: testing the hypothesis that the intercept is significantly different from the risk-free rate
        # Hint: F-test

        
    return 

In [ ]:
# Call the function portfolio_factor_sensitivity for the equally weighted portfolio


In [ ]:
# Call the function portfolio_factor_sensitivity for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_factor_sensitivity for minimum volatility portfolio


### 4. Regress the portfolio return on all factors and assess the portfolio's sensitivity to factors.

#### For the curious, optional challenge, how do you test whether the intercept (i.e., alpha) is significantly different from the risk-free rate for a multi-factor regression?
#### Hint: Write functions for repetitive lines of codes

In [ ]:
# Import the ols function
import statsmodels.api as sm

# function that takes portfolio and factor returns and run a regression of portfolio return on a return factor
# it reports the portfolio sensitivity to a return factor
def portfolio_all_factor_sensitivity(portfolio_type, df):
    
    ''' 
    INPUTS
    portfolio type: portfolio_ew, portfolio_maxsharpe, portfolio_minvol
    df: dataframe containing portfolio and factor returns
    
    OUTPUTS
    regression result of a portfolio on a return factor
    
    '''    
        
    # Create the model object

    
    # Fit the model

    # Print the parameters of the fitted model
    
    # Optional challenge: testing the hypothesis that the intercept is significantly different from the risk-free rate
    # Hint: F-test


    return 

In [ ]:
# Call the function portfolio_all_factor_sensitivity for the equally weighted portfolio


In [ ]:
# Call the function portfolio_all_factor_sensitivity for the maximum Sharpe portfolio


In [ ]:
# Call the function portfolio_all_factor_sensitivity for the minimum volatility portfolio


### 5. Optional Bonus. Construct a multi-factor pricing model for assets based on Arbitrage Pricing Theory.

#### The Arbitrage Pricing Theory (APT) is a theory of asset pricing that holds that an asset’s returns can be forecasted with the linear relationship of an asset’s expected returns and the macroeconomic (e.g., GDP, changes in inflation, yield curve changes, changes in interest rates, market sentiments, exchange rates) or firm-specific statistical factors that affect the asset’s risk. Hint: You can draw these variables straight into your Jupyter notebook via Refinitiv API.

#### The APT is a substitute for the Capital Asset Pricing Model (CAPM) in that both assert a linear relation between assets’ expected returns and their covariance with other random variables. (In the CAPM, the covariance is with the market portfolio’s return.) The covariance is interpreted as a measure of risk that investors cannot avoid by diversification. The slope coefficient in the linear relation between the expected returns and the covariance is interpreted as a risk premium ~ "Arbitrage Pricing Theory (Guberman and Wang 2005).

Acknowledgement: This notebook is inspired by DataCamp course "Introduction to Portfolio Analysis in Python" by Charlotte Werger.